In [ ]:
max_volume = 100

unit_profit = [20, 5, 10, 40, 15, 25]

risk = [
    [0.04, 0.028, 0.036, 0.03, 0.034, 0.032],
    [0.028, 0.04, 0.026, 0.034, 0.03, 0.036],
    [0.036, 0.026, 0.04, 0.032, 0.028, 0.038],
    [0.03, 0.034, 0.032, 0.04, 0.026, 0.036],
    [0.034, 0.03, 0.028, 0.026, 0.04, 0.032],
    [0.032, 0.036, 0.038, 0.036, 0.032, 0.04]
]

distance = [
    [5, 10, 15, 20, 25, 30],
    [10, 5, 20, 15, 30, 25],
    [15, 20, 5, 30, 10, 35],
    [20, 15, 30, 5, 25, 10],
    [25, 30, 10, 25, 5, 15],
    [30, 25, 35, 10, 15, 5]
]

is_abroad = [0, 1, 0, 1, 0, 1]


# Available stock for each type of product, rows represent products and columns represent different suppliers
# Odd columns are all 0 since they represent storages, not suppliers. As is_abroad indicates.
stock = [
    [0, 230, 0, 75, 0, 95],
    [0, 105, 0, 155, 0, 175],
    [0, 185, 0, 130, 0, 115],
    [0, 165, 0, 110, 0, 135],
    [0, 140, 0, 205, 0, 160],
    [0, 175, 0, 135, 0, 90],
    [0, 90, 0, 120, 0, 170],
    [0, 220, 0, 180, 0, 150],
    [0, 135, 0, 155, 0, 175],
    [0, 115, 0, 195, 0, 145],
    [0, 150, 0, 130, 0, 100],
    [0, 120, 0, 140, 0, 200]
]

volume_per_product = [10, 20, 15, 25, 30, 12, 18, 22, 14, 28, 16, 24] 

# Maximum storage capacity per location (0 for suppliers)
max_volume_storage_per_location = [3000, 0, 2500, 0, 2000, 0]

max_risk = 0.3

In [ ]:
from typing import Any, List, Optional
from dataclasses import dataclass
import heapq


@dataclass
class Node:
    """Represents a node in the search tree."""
    level: int  # Depth in the tree
    bound: float  # Upper bound on the optimal solution
    value: float  # Current value of this partial solution
    path: List[Any]  # Decisions made so far
    data: dict  
    
    def __lt__(self, other):
        # For max problems, use -bound; for min problems, use bound
        return self.bound > other.bound  # Assumes maximization



In [ ]:
# Auxiliar to find visited locations from adjacency matrix

def get_visited(matrix):
    n = len(matrix)
    indices = set()

    # Check rows
    for i in range(n):
        if 1 in matrix[i]:
            indices.add(i)  # 0-based index

    # Check columns
    for j in range(n):
        if any(matrix[i][j] == 1 for i in range(n)):
            indices.add(j)  # 0-based index

    # Remove 0 if present
    indices.discard(0)

    return indices

In [ ]:

class BranchAndBound:
    """General Branch and Bound algorithm structure."""
    
    from typing import List

    def __init__(
        self,
        max_volume: int,
        unit_profit: List[int],
        risk: List[List[float]],
        distance: List[List[int]],
        is_abroad: List[int],
        stock: List[List[int]],
        volume_per_product: List[int],
        max_volume_storage_per_location: List[int],
        max_risk: float
    ) -> None:
    
        self.max_volume = max_volume
        self.unit_profit = unit_profit
        self.risk = risk
        self.distance = distance
        self.is_abroad = is_abroad
        self.stock = stock
        self.volume_per_product = volume_per_product
        self.max_volume_storage_per_location = max_volume_storage_per_location
        self.max_risk = max_risk

        self.item_amount = len(stock)  # Number of different product types
        self.location_amount = len(is_abroad)
        self.best_value = float('-inf') 
        self.best_solution = None
        self.nodes_explored = 0

        
    def solve(self) -> tuple[Optional[Any], float]:
        """
        Main Branch and Bound algorithm.
        
        Returns:
            tuple: (best_solution, best_value)
        """
        # Initialize with root node
        root = self.create_root_node()
        
        # Priority queue for nodes to explore (using heap)
        queue = []
        heapq.heappush(queue, root)
        
        while queue:
            # Get the most promising node
            current_node = heapq.heappop(queue)
            self.nodes_explored += 1
            
            # Prune if bound is worse than best found
            if not self.should_explore(current_node):
                continue
            
            # Check if we have a complete solution
            if self.is_complete_solution(current_node):
                self.update_best_solution(current_node)
                continue
            
            # Branch: generate child nodes
            children = self.branch(current_node)
            
            for child in children:
                # Check feasibility
                if not self.is_feasible(child):
                    continue
                
                # Bound: calculate upper/ bound
                child.bound = self.compute_bound(child)
                
                # Add to queue if promising
                if self.should_explore(child):
                    heapq.heappush(queue, child)
        
        return self.best_solution, self.best_value

    
    def create_root_node(self) -> Node:
        """
        Create the initial root node of the search tree.
        
        Returns:
            Node: The root node representing the empty solution
        """
        return Node(
            level=0,
            bound=self.compute_initial_bound(),
            value=0,
            path=[],
            data={
                    'current_location': 0,
                    'current_volume': 0,  # Current volume in truck
                    'travels_from_i_to_j': [
                        [0 for j in range(self.location_amount)]
                        for i in range(self.location_amount)
                    ],

                    'items_taken_from_supplier': [
                        [0 for i in range(self.location_amount)]
                        for k in range(self.item_amount)
                    ],

                    'items_left_at_location': [
                        [0 for i in range(self.location_amount)]
                        for k in range(self.item_amount)
                    ],
                    
                    'remaining_stock': [
                        [self.stock[k][i] for i in range(self.location_amount)]
                        for k in range(self.item_amount)
                    ],
                    
                    # Track what's currently in the truck for each item
                    'items_in_truck': [0] * self.item_amount
                }
        )
        
    def branch(self, node: Node) -> List[Node]:
        """
        Generate child nodes by branching from the current node.
        
        The decision has two parts:
        1. Choose next location to visit
        2. If location is abroad (supplier): decide how much of each item to pick up
           If location is storage: decide how much of each item to leave
        
        Key insight: When picking up at a supplier, we DON'T know yet if items will be
        delivered. The bound provides optimistic guidance, but actual profit is only
        realized when items are confirmed left at storage in a complete solution.
        
        Args:
            node: Current node to branch from
            
        Returns:
            List[Node]: List of child nodes representing different decisions
        
        """
        
        children = []
        current_location = node.data['current_location']
        visited = get_visited(node.data['travels_from_i_to_j'])

        for next_location in range(self.location_amount):
            if next_location != current_location and next_location not in visited:
                
                if self.is_abroad[next_location] == 1:
                    # Supplier location: generate children for different pickup combinations
                    children.extend(self._branch_supplier(node, current_location, next_location))
                else:
                    # Storage location: generate children for different dropoff combinations
                    children.extend(self._branch_storage(node, current_location, next_location))
        
        return children
    
    def _branch_supplier(self, node: Node, current_location: int, next_location: int) -> List[Node]:
        """
        Branch at a supplier location: decide how much of each item to pick up.
        
        Important: At this point, we're making a pickup decision WITHOUT knowing for certain
        if these items will be delivered. The bound calculation provides optimistic guidance,
        but the actual profit depends on future decisions (whether to drop off or not).
        
        Args:
            node: Current node
            current_location: Current location index
            next_location: Supplier location index
            
        Returns:
            List[Node]: Child nodes with different pickup combinations
        """
        children = []
        
        # Option 1: Try to pick up as much as possible (greedy approach)
        new_data = self._create_child_data(node, current_location, next_location)
        current_volume = node.data['current_volume']
        remaining_capacity = self.max_volume - current_volume
        
        pickup_quantities = [0] * self.item_amount
        
        for item_idx in range(self.item_amount):
            available = node.data['remaining_stock'][item_idx][next_location]
            item_volume = self.volume_per_product[item_idx]
            
            if item_volume > 0:
                # Pick up as many as possible
                max_pickup = min(available, remaining_capacity // item_volume)
                pickup_quantities[item_idx] = max_pickup
                
                new_data['items_taken_from_supplier'][item_idx][next_location] += max_pickup
                new_data['remaining_stock'][item_idx][next_location] -= max_pickup
                new_data['items_in_truck'][item_idx] += max_pickup
                new_data['current_volume'] += max_pickup * item_volume
                remaining_capacity -= max_pickup * item_volume
        
        # Value remains 0 because we don't know yet if these will be delivered
        child_node = Node(
            level=node.level + 1,
            bound=0,
            value=0,  # No profit from just picking up
            path=node.path + [(next_location, 'pickup', pickup_quantities)],
            data=new_data
        )
        children.append(child_node)
        
        # Option 2: Skip this supplier (pick up nothing)
        new_data_skip = self._create_child_data(node, current_location, next_location)
        child_node_skip = Node(
            level=node.level + 1,
            bound=0,
            value=0,
            path=node.path + [(next_location, 'skip', [0] * self.item_amount)],
            data=new_data_skip
        )
        children.append(child_node_skip)
        
        return children
    
    def _branch_storage(self, node: Node, current_location: int, next_location: int) -> List[Node]:
        """
        Branch at a storage location: decide how much of each item to leave.
        
        This is where uncertainty gets resolved: by choosing to drop off items here,
        we're committing that these items will contribute to profit (assuming we
        complete the route successfully).
        
        Args:
            node: Current node
            current_location: Current location index
            next_location: Storage location index
            
        Returns:
            List[Node]: Child nodes with different dropoff combinations
        """
        children = []
        
        # What we're currently carrying
        current_items = node.data['items_in_truck'].copy()
        
        # Option 1: Leave everything at this storage
        new_data = self._create_child_data(node, current_location, next_location)
        dropoff_quantities = current_items.copy()
        
        for item_idx in range(self.item_amount):
            new_data['items_left_at_location'][item_idx][next_location] = dropoff_quantities[item_idx]
            new_data['items_in_truck'][item_idx] = 0
        
        # Update volume (now empty)
        new_data['current_volume'] = 0
        
        # Still don't calculate profit - only at completion
        # This allows for flexibility if we need to pick items back up (though unusual)
        child_node = Node(
            level=node.level + 1,
            bound=0,
            value=0,  # Profit calculated only at completion
            path=node.path + [(next_location, 'dropoff', dropoff_quantities)],
            data=new_data
        )
        children.append(child_node)
        
        # Option 2: Keep everything (just visit, don't drop off)
        # This is a valid decision: we visited but chose not to deliver here
        new_data_keep = self._create_child_data(node, current_location, next_location)
        child_node_keep = Node(
            level=node.level + 1,
            bound=0,
            value=0,
            path=node.path + [(next_location, 'keep', [0] * self.item_amount)],
            data=new_data_keep
        )
        children.append(child_node_keep)
        
        # Option 3: Partial dropoff (can be expanded for more granular decisions)
        # For now, we keep it simple with all-or-nothing
        
        return children
    
    def _create_child_data(self, node: Node, current_location: int, next_location: int) -> dict:
        """
        Create a copy of node data with updated travel information.
        
        Args:
            node: Parent node
            current_location: Current location
            next_location: Next location
            
        Returns:
            dict: New data dictionary for child node
        """
        new_data = {
            'current_location': next_location,
            'current_volume': node.data['current_volume'],
            'travels_from_i_to_j': [row.copy() for row in node.data['travels_from_i_to_j']],
            'items_taken_from_supplier': [row.copy() for row in node.data['items_taken_from_supplier']],
            'items_left_at_location': [row.copy() for row in node.data['items_left_at_location']],
            'remaining_stock': [row.copy() for row in node.data['remaining_stock']],
            'items_in_truck': node.data['items_in_truck'].copy()
        }
        
        # Update travels
        new_data['travels_from_i_to_j'][current_location][next_location] = 1
        
        return new_data
    
    def _calculate_total_profit(self, node: Node) -> float:
        """
        Calculate the total profit based on items left at storage locations.
        Only items that remain at storage locations at the end generate profit.
        
        This is called ONLY when we have a complete solution (returned to depot).
        At that point, we know with certainty which items were delivered.
        
        Args:
            node: Node to calculate profit for
            
        Returns:
            float: Total profit value
        """
        total_profit = 0.0
        
        # Only count items left at storage locations (is_abroad == 0)
        for location_idx in range(self.location_amount):
            if self.is_abroad[location_idx] == 0:  # Storage location
                for item_idx in range(self.item_amount):
                    quantity = node.data['items_left_at_location'][item_idx][location_idx]
                    profit_per_unit = self.unit_profit[item_idx]
                    total_profit += quantity * profit_per_unit
        
        return total_profit

    
    def is_feasible(self, node: Node) -> bool:
        """
        Check if the node represents a feasible partial solution.
        
        Args:
            node: Node to check
            
        Returns:
            bool: True if feasible, False otherwise
        """
        # Check volume constraint
        if node.data['current_volume'] > self.max_volume:
            return False
        
        # Verify items_in_truck matches current_volume
        calculated_volume = sum(
            node.data['items_in_truck'][i] * self.volume_per_product[i]
            for i in range(self.item_amount)
        )
        if abs(calculated_volume - node.data['current_volume']) > 0.01:
            return False
        
        # Check storage capacity at each location
        for location_idx in range(self.location_amount):
            if self.is_abroad[location_idx] == 0:  # Storage location
                total_volume = sum(
                    node.data['items_left_at_location'][item_idx][location_idx] * 
                    self.volume_per_product[item_idx]
                    for item_idx in range(self.item_amount)
                )
                if total_volume > self.max_volume_storage_per_location[location_idx]:
                    return False

        # Check risk constraint
        total_risk = 0.0
        for i in range(self.location_amount):
            for j in range(self.location_amount):
                total_risk *= (1 - node.data['travels_from_i_to_j'][i][j] * self.risk[i][j])
        
        total_risk = 1 - total_risk
        
        if total_risk > self.max_risk:
            return False
        
        return True
        
    def compute_initial_bound(self) -> float:
        """Compute initial optimistic bound."""
        # Simple upper bound: sum of all possible profits
        return sum(sum(row) for row in self.stock) * max(self.unit_profit)
    
    def compute_bound(self, node: Node) -> float:
        """
        Compute an optimistic bound for the node.
        
        This is KEY to handling uncertainty: the bound assumes the BEST CASE scenario
        where all picked-up items eventually get delivered, and all remaining stock
        can still be picked up and delivered.
        
        Args:
            node: Node to compute bound for
            
        Returns:
            float: The computed bound (optimistic estimate)
        """
        # Profit from items already delivered (certain)
        current_profit = self._calculate_total_profit(node)
        
        # Optimistic value from items currently in truck
        # (assumes they WILL be delivered)
        truck_value = sum(
            node.data['items_in_truck'][i] * self.unit_profit[i % len(self.unit_profit)]
            for i in range(self.item_amount)
        )
        
        # Optimistic value from remaining stock
        # (assumes we CAN pick up and deliver everything)
        remaining_value = 0
        for item_idx in range(self.item_amount):
            remaining = sum(node.data['remaining_stock'][item_idx])
            profit_per_unit = self.unit_profit[item_idx % len(self.unit_profit)]
            remaining_value += remaining * profit_per_unit
        
        return current_profit + truck_value + remaining_value
    
    def is_complete_solution(self, node: Node) -> bool:
        """
        Check if the node represents a complete solution.
        
        Args:
            node: Node to check
            
        Returns:
            bool: True if complete, False otherwise
        """
        # Complete when truck is back at depot (location 0) with empty cargo
        # and has visited at least some locations
        return (node.data['current_location'] == 0 and 
                node.level > 0 
                #and 
                # node.data['current_volume'] == 0 and
                # sum(node.data['items_in_truck']) == 0
                )
    
    def should_explore(self, node: Node) -> bool:
        """
        Determine if a node should be explored (pruning condition).
        
        Args:
            node: Node to check
            
        Returns:
            bool: True if should explore, False if should prune
        """
        # For maximization
        return node.bound > self.best_value
    
    def update_best_solution(self, node: Node) -> None:
        """
        Update the best solution found so far if this node is better.
        
        This is where uncertainty is RESOLVED: we have a complete route,
        we know exactly which items were delivered, so we can calculate
        the actual profit.
        
        Args:
            node: Complete solution node to compare
        """
        # Calculate actual profit from items left at storage
        actual_profit = self._calculate_total_profit(node)
        
        if actual_profit > self.best_value:  # For maximization
            self.best_value = actual_profit
            self.best_solution = node.path.copy()



In [ ]:
problem = BranchAndBound(
    max_volume,
    unit_profit,
    risk,
    distance,
    is_abroad,
    stock,
    volume_per_product,
    max_volume_storage_per_location,
    max_risk
)

best_solution, best_value = problem.solve()